In [2]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from sklearn import svm
from sklearn.feature_selection import mutual_info_regression
import itertools as it
import pickle
import sys, copy
from sklearn.linear_model import LogisticRegression as logreg

In [3]:
# dataset = "gisette"
# dataset = "spambase"
dataset = "synth"
# dataset = "speed"
# dataset = "spo2"
# dataset = "test"

# test thetas mi
# gisette: 0.12
# spambase: 0.1
# synth: 0.3
# speed: None
# spo2: 0.1

thetas = {"gisette": 0.12, "spambase": 0.1, "synth": 0.3, "speed": None, "spo2": 0.1, "test": 0.5}

if dataset=="gisette":

    fileX = "./data/gisette/GISETTE/gisette_train.data"
    filey = "./data/gisette/GISETTE/gisette_train.labels"

    dfX = pd.read_csv(fileX, sep=' ', header=None)
    X = np.array(dfX)[:,:-1]
    dfy = pd.read_csv(filey, sep=' ', header=None)
    y = np.array(dfy)

if dataset=="spambase":

    X = pickle.load(open("./data/spambase/X_spambase.p", 'rb')).to_numpy()
    y = pickle.load(open("./data/spambase/y_spambase.p", 'rb')).to_numpy()
    
    # normalizing
    for ii in range(X.shape[1]):
        X[:,ii] = X[:,ii] - X[:,ii].mean()
        X[:,ii] = X[:,ii] / (4*X[:,ii].std())

if dataset == "synth":
    X = np.random.uniform(low=-1, high=1, size=[20000,300])
    y = X[:,0]*X[:,1]
    y = y.reshape(y.shape[0],1)

if dataset=="speed":
    fileX = "./data/speed/Input-P0-0-SD-DATA"
    filey = "./data/speed/Input-P1-0-SD-LABEL-CLASS"

    dfX = pd.read_csv(fileX, sep=' ', header=None)
    X = np.array(dfX)

    dfy = pd.read_csv(filey, sep=' ', header=None)
    y = np.array(dfy)

    # last two features are matching features like the target
    X = X[:, :120]

if dataset=="spo2":
    
    # not checked in this data
    # copy these files to the spo2 folder
    fileX = "./data/spo2/Xdata_hist.txt"
    filey = "./data/spo2/ydata_hist.txt"

    dfX = pd.read_csv(fileX, sep=' ', header=None)
    X = np.array(dfX)

    dfy = pd.read_csv(filey, sep=' ', header=None)
    y = np.array(dfy)

if dataset=="test":
    N = 10000
    X = np.random.uniform(low=-1, high=1, size=[N,10])
    y = np.sign(X[:,0])*np.random.choice([-1,1], N, p=[0.1, 0.9])
    y = y.reshape(y.shape[0],1)
    
def split_data(X,y):
    indecies_train = np.random.choice(range(X.shape[0]), round(0.8*X.shape[0]), replace=False)
    indecies_test = np.array([ii for ii in range(X.shape[0]) if ii not in indecies_train])

    X_train = X[indecies_train,:]
    y_train = y[indecies_train,:]
    X_test = X[indecies_test,:]
    y_test = y[indecies_test,:]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = split_data(X,y)

In [4]:
def MI(xin, yin, binsx, binsy, rangex, rangey, use_bits=True, PRINT=False):    
    N = xin.shape[0]
    H, xe, ye = np.histogram2d(xin, yin.reshape(yin.shape[0]), bins=[binsx, binsy], range=[rangex, rangey])
    H = H/N
    N = X.shape[0]
    Hx = H.sum(axis=1)
    Hy = H.sum(axis=0)
    if use_bits:
        Ex = np.array([-x*np.log2(x) for x in Hx if x > 0]).sum()
        Ey = np.array([-x*np.log2(x) for x in Hy if x > 0]).sum()
        Exy = np.array([-x*np.log2(x) for x in H.flatten() if x > 0]).sum()
    else:
        Ex = np.array([-x*np.log(x) for x in Hx if x > 0]).sum()
        Ey = np.array([-x*np.log(x) for x in Hy if x > 0]).sum()
        Exy = np.array([-x*np.log(x) for x in H.flatten() if x > 0]).sum()           
    I = Ex+Ey-Exy
    if PRINT:
        print(Ex,Ey,Exy,I)
    return Ex,Ey,Exy,I

def specbound3(xin1, xin2, xin3, bins=10, rangex=[-1,1], use_bits=True):
    cutoff = 1e-4

    N1 = xin1.shape[0]
    N2 = xin2.shape[0]
    N3 = xin3.shape[0]
    N = N1 + N2 + N3
    
    H1, xe1, = np.histogram(xin1, bins=bins, range=rangex)
    H2, xe2, = np.histogram(xin2, bins=bins, range=rangex)
    H3, xe3, = np.histogram(xin3, bins=bins, range=rangex)
    H1 = H1/N1
    H2 = H2/N2
    H3 = H3/N3
    
    H1 = np.array([max(h,cutoff) for h in H1])
    H2 = np.array([max(h,cutoff) for h in H2])
    H3 = np.array([max(h,cutoff) for h in H3])
    
    w1 = len(xin1)/N
    w2 = len(xin2)/N
    w3 = len(xin3)/N
    
    if use_bits:
        # Exspec = np.array([-(w1*x1+w2*x2+w3*x3)*(w1*np.log2(x1c)+w2*np.log2(x2c)+w3*np.log2(x3c)) for x1,x2,x3,x1c,x2c,x3c in zip(H1,H2,H3,H1c,H2c,H3c)]).sum()
        Exspec = np.array([-(w1*x1+w2*x2+w3*x3)*(w1*np.log2(x1)+w2*np.log2(x2)+w3*np.log2(x3)) for x1,x2,x3 in zip(H1,H2,H3)]).sum()
    else:
        # Exspec = np.array([-(w1*x1+w2*x2+w3*x3)*(w1*np.log(x1c)+w2*np.log(x2c)+w3*np.log(x3c)) for x1,x2,x3,x1c,x2c,x3c in zip(H1,H2,H3,H1c,H2c,H3c)]).sum()
        Exspec = np.array([-(w1*x1+w2*x2+w3*x3)*(w1*np.log(x1)+w2*np.log(x2)+w3*np.log(x3)) for x1,x2,x3 in zip(H1,H2,H3)]).sum()
    return Exspec


def mi_bound(Xin, yin, binsx, binsy, rangex, rangey, use_bits=True, test=False):
    N = Xin.shape[0]
    if not test:
        ind = np.random.permutation(N)
        ind1 = ind[:round(N/3)]
        ind2 = ind[round(N/3):round(2*N/3)]
        ind3 = ind[round(2*N/3):]
    else:
        ind = [ii for ii in range(N) if yin[ii]==np.sign(Xin[ii,1])==-1] + [ii for ii in range(N) if yin[ii]==np.sign(Xin[ii,1])==1] + [ii for ii in range(N) if yin[ii]==-np.sign(Xin[ii,1])]
        ind1 = ind[:round(N/4)]
        ind2 = ind[round(N/4):round(N/2)]
        ind3 = ind[round(N/2):]

    res = []
    for ii in range(Xin.shape[1]):
        
        Exf, Eyf, Exyf, If = MI(X[:,ii], y, binsx, binsy, rangex, rangey, use_bits=use_bits)
        #print()
        
        Xs1 = Xin[ind1,ii]
        Xs2 = Xin[ind2,ii]
        Xs3 = Xin[ind3,ii]   
        ys1 = yin[ind1]
        ys2 = yin[ind2]
        ys3 = yin[ind3]

        Ex1, Ey1, Exy1, I1 = MI(Xs1, ys1, binsx, binsy, rangex, rangey, use_bits=use_bits)
        Ex2, Ey2, Exy2, I2 = MI(Xs2, ys2, binsx, binsy, rangex, rangey, use_bits=use_bits)
        Ex3, Ey3, Exy3, I3 = MI(Xs3, ys3, binsx, binsy, rangex, rangey, use_bits=use_bits)
        #print()

        # lower bound on entropy of joint variable
        bound = Exy1*len(ind1)/N+Exy2*len(ind2)/N+Exy3*len(ind3)/N
        
        Exspec = specbound3(Xs1, Xs2, Xs3, bins=binsx, rangex=rangex, use_bits=use_bits)

        res.append(Exspec + Eyf - bound)
    
    return res

In [6]:
# thetas["test"] = 0.4

def mi_naive_ith(xx, yy, binsx, binsy, rangex, rangey, use_bits=True):
    N = xx.shape[0]

    H, xe, ye = np.histogram2d(xx, yy.flatten(), bins=[binsx, binsy], range=[rangex,rangey])
    Hx = H.sum(axis=1)
    Hy = H.sum(axis=0)

    out = 0
    for i,j in it.product(range(H.shape[0]), range(H.shape[1])):
        if H[i,j] == 0:
            continue
        
        # actual mi
        if use_bits:
            out += H[i,j]*np.log2(H[i,j]*N/(Hx[i]*Hy[j]))/N
        else:
            out += H[i,j]*np.log(H[i,j]*N/(Hx[i]*Hy[j]))/N
    
    return out

def mi_naive(XX, yy, binsx, binsy, rangex, rangey, use_bits=True):
    return [mi_naive_ith(x, yy, binsx, binsy, rangex, rangey, use_bits=use_bits) for x in XX.transpose()]

def train_svm(Xl_train, yl_train, Xl_test, yl_test):
    clf = svm.SVC()
    clf.fit(Xl_train, yl_train.reshape(yl_train.shape[0],))

    pred_train = clf.predict(Xl_train)
    acc_train = (pred_train == yl_train.flatten().reshape(yl_train.shape[0])).astype(int).sum()/yl_train.shape[0]

    pred_test = clf.predict(Xl_test)
    acc_test = (pred_test == yl_test.flatten().reshape(yl_test.shape[0])).astype(int).sum()/yl_test.shape[0]
    
    print("acc_train:", acc_train)
    print("acc_test:", acc_test)
    
    return acc_train, acc_test

def train_logreg(Xl_train, yl_train, Xl_test, yl_test):
    
    model = logreg(max_iter=5000)
    model.fit(Xl_train, yl_train.reshape(yl_train.shape[0],))

    pred_train = model.predict(Xl_train)
    acc_train = (pred_train == yl_train.reshape(yl_train.shape[0])).astype(int).sum()/yl_train.shape[0]

    pred_test = model.predict(Xl_test)
    acc_test = (pred_test == yl_test.reshape(yl_test.shape[0])).astype(int).sum()/yl_test.shape[0]

    print("acc_train:", acc_train)
    print("acc_test:", acc_test)
    
    return acc_train, acc_test

def train_reg_model(Xl_train, yl_train, Xl_test, yl_test):   
    clf = svm.SVR()
    clf.fit(Xl_train, yl_train.reshape(yl_train.shape[0],))

    pred_train = clf.predict(Xl_train)
    pred_test = clf.predict(Xl_test)
    
    rmse_train = np.sqrt(((pred_train - yl_train.flatten().reshape(yl_train.shape[0]))**2).mean())
    rmse_test = np.sqrt(((pred_test - yl_test.flatten().reshape(yl_test.shape[0]))**2).mean())
    
    print("rmse train:", rmse_train)
    print("rmse test:", rmse_test)
    
    return rmse_train, rmse_test
    
    
def select_feat(Xl_train, y_train, Xl_test, y_test, do_all=False, sel="random", theta=None, test=False):
    
    if do_all:
        return Xl_train, Xl_test, "all"
    
    if dataset=="gisette":
        nfeat = 100
        binsx = 30
        binsy = 2
        rangex = [0,1000]
        rangey = [-1,1]
    if dataset=="spambase":
        nfeat = 10
        binsx = 10
        binsy = 2
        rangex = [-1,1]
        rangey = [0,1]
    if dataset=="synth":
        nfeat = 2
        binsx = 35
        binsy = 35
        rangex = [-1,1]
        rangey = [-1,1]
    if dataset=="speed":
        nfeat = 20
        binsx = 30
        binsy = 2
        # fix this 
        # not using speed dataset
        # rangex = [0,1]
        rangex = [-1,258]
        rangey = [0,1]
    if dataset=="spo2":
        nfeat = 7
        binsx = 20
        binsy = 2
        rangex = [0,300]
        rangey = [0,1]
    if dataset=="test":
        nfeat= 1
        binsx = 35
        binsy = 35
        rangex = [-1,1]
        rangey = [-1,1]
    
    if theta != None:
        print("Using threshold %s for filtering." % theta)
    else:
        print("Using default number of features %s." % nfeat)
    
    if sel == "random":
        assert theta == None, "Random selection not compatible with providing threshold."
        selected = np.random.choice(range(Xl_train.shape[1]), size=nfeat, replace=False)
        print(selected)
        return Xl_train[:,selected], Xl_test[:,selected], selected
    
    if sel == "corr":
        selected_scores = []
        for ii in range(Xl_train.shape[1]):
            cc = np.corrcoef(Xl_train[:,ii], y_train[:,0])
            selected_scores.append(np.abs(cc[0][1]))
        if theta != None:
            nfeat = len([c for c in selected_scores if c >= theta])
        the_args = np.argsort(selected_scores)
        selected = np.flip(the_args)[0:nfeat]
        print(selected)
        # print(selected_scores)
        return Xl_train[:,selected], Xl_test[:,selected], selected
    
    if sel == "mi":
        mi = mutual_info_regression(Xl_train, y_train)
        if theta != None:
            nfeat = len([c for c in mi if c >= theta])
        mi_args = np.argsort(mi)
        selected = np.flip(mi_args)[0:nfeat]
        print(selected)
        # print(mi)
        return Xl_train[:,selected], Xl_test[:,selected], selected
    
    if sel == "mi_naive":
        mi = mi_naive(Xl_train, y_train, binsx, binsy, rangex, rangey)
        if theta != None:
            nfeat = len([c for c in mi if c >= theta])
        mi_args = np.argsort(mi)
        selected = np.flip(mi_args)[0:nfeat]
        # print(mi)
        print(selected)
        print()
        print("nfeat: ", nfeat)
        print()
        #for mm in np.flip(sorted(mi)):
        #    print(mm)
        #for mm in mi:
        #    print(mm)
        return Xl_train[:,selected], Xl_test[:,selected], selected
    
    if sel == "mi_bound":
        mi = mi_bound(Xl_train, y_train, binsx, binsy, rangex, rangey, test=test)
        if theta != None:
            nfeat = len([c for c in mi if c >= theta])
        mi_bound_args = np.argsort(mi)
        selected = np.flip(mi_bound_args)[0:nfeat]
        # print(mi)
        print(selected)
        print()
        print("nfeat: ", nfeat)
        print()
        #for mm in np.flip(sorted(mi)):
        #    print(mm)
        for mm in mi:
            print(mm)
        return Xl_train[:,selected], Xl_test[:,selected], selected

    
    if sel == "mi_mix":
        length = X_train.shape[0]
        
        if not test:
            indecies = np.random.permutation(length)
            rows0 = indecies[0:round(length/3)]
            rows1 = indecies[round(length/3):round(2*length/3)]
            rows2 = indecies[round(2*length/3):]
        else:
            indecies = [ii for ii in range(length) if (y_train[ii]==np.sign(X_train[ii,1])==-1)] + [ii for ii in range(length) if (y_train[ii]==np.sign(X_train[ii,1])==1)]
            indecies += [ii for ii in range(length) if y_train[ii]==-np.sign(X_train[ii,1])]
            rows0 = indecies[0:round(length/4)]
            rows1 = indecies[round(length/4):round(length/2)]
            rows2 = indecies[round(length/2):]
        
        X0 = X_train[rows0,:]
        X1 = X_train[rows1,:]
        X2 = X_train[rows2,:]
        y0 = y_train[rows0]
        y1 = y_train[rows1]
        y2 = y_train[rows2]
            
        mi_mix0 = np.array(mi_naive(X0, y0, binsx, binsy, rangex, rangey))
        mi_mix1 = np.array(mi_naive(X1, y1, binsx, binsy, rangex, rangey))
        mi_mix2 = np.array(mi_naive(X2, y2, binsx, binsy, rangex, rangey))
        mi_mix = (len(rows0)*mi_mix0 + len(rows1)*mi_mix1 + len(rows2)*mi_mix2)/length
        
        if theta != None:
            nfeat = len([c for c in mi_mix if c >= theta])

        mi_mix_args = np.argsort(mi_mix)
        selected = np.flip(mi_mix_args)[0:nfeat]
        print(selected)
        print(mi_mix)
        return Xl_train[:,selected], Xl_test[:,selected], selected
    
    if sel == "var":
        var_train = np.var(X_train, axis=0)
        if theta != None:
            nfeat = len([c for c in var_train if c >= theta])
        var_args = np.argsort(var_train)
        selected = np.flip(var_args)[0:nfeat]
        print(selected)
        print(var_train)
        return Xl_train[:,selected], Xl_test[:,selected], selected

    return Xl_train[:,selected], Xl_test[:,selected], selected

test = dataset=="test"
numrounds = 1
res = []
for rr in range(numrounds):
    # X_train, y_train, X_test, y_test = split_data(X,y)
    
    Xs_train, Xs_test, selected = select_feat(X_train, y_train, X_test, y_test, do_all=False, sel="mi_naive", theta=thetas[dataset], test=test)
    
    if dataset in ["gisette", "spo2", "test"]:
        acc_train, acc_test = train_svm(Xs_train, y_train, Xs_test, y_test)
        res.append(acc_test)

    if dataset in ["speed", "spambase"]:
        acc_train, acc_test = train_logreg(Xs_train, y_train, Xs_test, y_test)
        res.append(acc_test)
    
    if dataset in ["synth"]:
        rmse_train, rmse_test = train_reg_model(Xs_train, y_train, Xs_test, y_test)
        res.append(rmse_test)
    print()

print()
print(res)
print(np.array(res).mean())

Using threshold 0.3 for filtering.
[1 0]

nfeat:  2

rmse train: 0.04653991255249811
rmse test: 0.046232072990549974


[0.046232072990549974]
0.046232072990549974
